## DEVLAND + REACT

Lists files that are present in each folder as well as the latest files

In [1]:
import os
import glob
import pandas as pd
import subprocess
import logging
import datetime
from openpyxl.styles import PatternFill

In [2]:
directories = [
    (r'R:\RawData\Elite Star\\Devland\RAW FILES RECEIVED', r'C:\Users\tsello01\\Documents\\Data\\Devland Latest.xlsx'),
    (r'R:\RawData\Elite Star\React Stores\Client Download Folders', r'C:\Users\\tsello01\\Documents\\Data\React Latest.xlsx')
]

In [3]:
def get_file_list(directory):
    """Retrieve a list of all files in the directory and its subdirectories."""
    return glob.glob(os.path.join(directory, '**'), recursive=True)

In [4]:
def get_most_recent_file(directory, files):
    """Retrieve the most recent file in the directory."""
    return max(files, key=os.path.getmtime)

In [5]:
def delete_files(files):
    """Delete the specified files."""
    for file in files:
        try:
            os.remove(file)
        except Exception as e:
            logging.error(f"Error deleting file {file}: {e}")

In [6]:
def list_files(directory):
  file_paths = []
  previous_date = None
  previous_folder = None

  for root, _, filenames in os.walk(directory):
      for filename in filenames:
          relative_path = os.path.relpath(os.path.join(root, filename), directory)
          folder_name = os.path.dirname(relative_path)

          # Check if the filename matches the expected format
          if '_' in filename:
              date_str = filename.split('_')[-1].split('.')[0]
          elif '-' in filename:
              date_str = os.path.splitext(filename)[0].split('-')[-1]
          else:
              continue

          try:
              file_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()
          except ValueError:
              continue

          # Reset previous_date if a new folder is encountered
          if folder_name != previous_folder:
              previous_date = None
              previous_folder = folder_name

          day_diff = (file_date - previous_date).days if previous_date else 0
          week_diff = day_diff // 7
          file_paths.append([relative_path, folder_name, week_diff])
          previous_date = file_date

  df = pd.DataFrame(file_paths, columns=['File Path', 'Folder', 'Week Difference'])

  # Apply conditional formatting
  df.style.apply(lambda x: ['background-color: yellow' if val > 1 else '' for val in x['Week Difference']], axis=0)

  return df

In [7]:
def list_most_recent_files(directory):
  recent_files = []

  for root, dirs, filenames in os.walk(directory):
      if filenames:
          most_recent_file = max(filenames, key=lambda x: os.path.getmtime(os.path.join(root, x)))
          relative_path = os.path.relpath(os.path.join(root, most_recent_file), directory)
          folder_name = os.path.basename(root)
          recent_files.append([folder_name, relative_path])

  df = pd.DataFrame(recent_files, columns=['Folder Name', 'Most Recent File'])

  return df


In [8]:
try:
    for directory, output_file in directories:
        with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
            list_files(directory).to_excel(writer, sheet_name='All Files', index=False)
            list_most_recent_files(directory).to_excel(writer, sheet_name='Most Recent Files', index=False)

    if os.name == 'nt':  # Windows
        for _, output_file in directories:
            os.startfile(output_file)
    else: 
        for _, output_file in directories:
            subprocess.call(['open', output_file])

except PermissionError:
    print("Error: The file(s) is already open. Please close it and try again.")